This Book will used to Calaculate the total pregnaices and establish the protcol for multiple pregainces and for the future cohort

### 1.A Full Pregengancy is if the diagonosi coloumsn have at least have  one of the delivery codes 
### 2.Determine the total number of procedures for each patient.
##  3.First by identifying pregeancy events and checking the time intervals between consecutive diagnosis intervals greater than 210 days will be considered a separate event 

Step one find the total  pregeancy for each patient

In [18]:
## Libaries 
import pandas as pd
import warnings
from pandas._libs.tslibs.parsing import DateParseError
import os
import time


In [19]:
# Load delivery-related diagnosis codes
delivery_codes_path = r"Z:\chelsea\datalake\final_codes\delivery_codes.csv"
delivery_codes_df = pd.read_csv(delivery_codes_path)
delivery_codes = delivery_codes_df['dx_cd'].tolist()

# Load pregnancy-related diagnosis codes
pregnancy_codes_path = r"Z:\chelsea\datalake\final_codes\pregnancy_code.csv"
pregnancy_codes_df = pd.read_csv(pregnancy_codes_path)
pregnancy_codes = pregnancy_codes_df['dx_cd'].tolist()


In [25]:
def calculate_total_pregnancies(df: pd.DataFrame,
                                delivery_codes: list,
                                pregnancy_codes: list,
                                pat_id: str = 'pat_id',
                                from_dt_col: str = 'from_dt',
                                to_dt_col: str = 'to_dt',
                                total_pregs: str = 'total_pregnancies',
                                dts_pregs: str = 'dts_pregnancies'):
    """
    Function to calculate total pregnancies for each patient.
    This function reruns patients if multiple pregnancies are detected.
    Now checks both delivery and pregnancy-related diagnosis codes.
    """
    # Convert the date columns to datetime format
    df[from_dt_col] = pd.to_datetime(df[from_dt_col], errors='coerce')
    df[to_dt_col] = pd.to_datetime(df[to_dt_col], errors='coerce')

    # Sort by patient ID and diagnosis date
    df = df.sort_values(by=[pat_id, from_dt_col])

    # Initialize columns for storing pregnancy information
    df[total_pregs] = 0
    df[dts_pregs] = None

    # Identify if any diagnosis code is in the list of delivery or pregnancy codes
    diag_cols = [f'diag{i}' for i in range(1, 13)]  # diag1 ... diag12
    all_codes = set(delivery_codes) | set(pregnancy_codes)  # union of both lists
    df['any_diag_in_dx_codes'] = df[diag_cols].isin(all_codes).any(axis=1)

    # Filter only rows where diagnosis codes are present and dates are valid
    df = df[df['any_diag_in_dx_codes'] & df[from_dt_col].notnull()]

    # Process each patient group to determine pregnancies
    for patient_id, group in df.groupby(pat_id):
        selected_rows = group[group['any_diag_in_dx_codes']]  # rows with delivery/pregnancy codes
        preg_dt = []

        if not selected_rows.empty:
            # Take the first date as initial pregnancy
            first_preg = selected_rows.iloc[0][from_dt_col]
            preg_dt.append(first_preg)
            num_pregnancies = 1

            # Calculate intervals between diagnosis dates to detect new pregnancies
            intervals = selected_rows[from_dt_col].diff().dt.days
            new_pregnancies = intervals > 210
            num_pregnancies += new_pregnancies.sum()
            preg_dt.extend(selected_rows[new_pregnancies][from_dt_col])

            # Update DataFrame with results
            df.loc[df[pat_id] == patient_id, total_pregs] = num_pregnancies
            df.loc[df[pat_id] == patient_id, dts_pregs] = ','.join(map(str, preg_dt))

    # Drop the helper column
    df.drop(columns=['any_diag_in_dx_codes'], inplace=True)

    return df



# Function to process pregnancies for patients across chunks
def process_pregnancies(df: pd.DataFrame, pat_id: str = 'pat_id', total_pregs: str = "total_pregnancies", 
                        pat_id_p: str = "pat_id_p", del_dt: str = "delivery_dt", dts_pregs: str = 'dts_pregnancies'):
    result_df = pd.DataFrame()
    groups_pat_id = df.groupby(pat_id)
    for patient_id, group in groups_pat_id:
        number_pregnancies = int(group[total_pregs].max())
        for i in range(1, number_pregnancies + 1):
            group[pat_id_p] = group[pat_id] + '_' + str(i)
            group[del_dt] = group[dts_pregs].apply(lambda x: x.split(',')[i - 1] if pd.notnull(x) else None)

            result_df = pd.concat([result_df, group], ignore_index=True)
    return result_df

# Function for formating dates
def format_date_cols (data:pd.DataFrame, col: str):
    """This function converts a column to datetime format.

    Parameters
    ----------
    data : pd.DataFrame
        dataframe to be manipulated.
    col : str
        column to be converted to datetime format.

    Returns
    -------
    data : pd.DataFrame
        dataframe with the column converted to datetime format.
    """
    data[col] = pd.to_datetime(data[col], infer_datetime_format=True, format ='mixed', errors='coerce')
    return data

In [22]:
# Define input and output paths
input_file_path = r"Z:\chelsea\datalake\New_cohort\claims_usa_females.csv"
output_file_initial = r"Z:\chelsea\datalake\New_cohort\pregnancy_identified.csv"  
output_file_fixed = r"Z:\chelsea\datalake\New_cohort\pregnancy_corrected.csv"    
output_file_final = r"Z:\chelsea\datalake\New_cohort\pregnancy_processed.csv"    

chunksize = 100000
total_patients_identified = 0
complete_info_patients = 0
incomplete_info_patients = 0


In [ ]:
# Step 1: Process data to create pregnancy_identified.csv
warnings.filterwarnings('ignore')

chunksize = 100000  
processed_patients = set()

start_time = time.time()  
total_patients_identified = 0

# Read the CSV file in chunks
for chunk_idx, chunk_df in enumerate(pd.read_csv(input_file_path, dtype=str, chunksize=chunksize)):

    try:
        # Track start time for the current chunk
        chunk_start_time = time.time()

        # Convert date columns to datetime
        chunk_df['from_dt'] = pd.to_datetime(chunk_df['from_dt'], errors='coerce')
        chunk_df['to_dt'] = pd.to_datetime(chunk_df['to_dt'], errors='coerce')

        # Filter out rows where dates are not valid
        chunk_df = chunk_df[chunk_df['from_dt'].notnull() & chunk_df['to_dt'].notnull()]

        # Calculate pregnancies for the chunk
        chunk_df = calculate_total_pregnancies(chunk_df, delivery_codes, 'pat_id', 'from_dt', 'to_dt', 'total_pregnancies', 'dts_pregs')
        chunk_df.reset_index(drop=True, inplace=True)

        # Identify patients with multiple pregnancies for rerun
        count = chunk_df.groupby('pat_id').agg({'total_pregnancies': 'unique'})
        count = count.reset_index()
        mask = count['total_pregnancies'].apply(len) > 1
        pat_ids = count.loc[mask, 'pat_id'].tolist()

        # Reprocess patients with multiple pregnancies
        rerun_patients = chunk_df[chunk_df['pat_id'].isin(pat_ids)]
        rerun_patients = calculate_total_pregnancies(rerun_patients, delivery_codes, 'pat_id', 'from_dt', 'to_dt', 'total_pregnancies', 'dts_pregs')

        # Replace the original rows for these patients with the rerun results
        chunk_df = chunk_df[~chunk_df['pat_id'].isin(pat_ids)]
        chunk_df = pd.concat([chunk_df, rerun_patients])

        # Count patients identified in this chunk
        patients_in_chunk = chunk_df['pat_id'].nunique()
        total_patients_identified += patients_in_chunk

        # Save the processed chunk
        if chunk_idx == 0:
            chunk_df.to_csv(output_file_initial, mode='w', index=False)
        else:
            chunk_df.to_csv(output_file_initial, mode='a', index=False, header=False)

        # Track the elapsed time for the current chunk
        chunk_elapsed_time = time.time() - chunk_start_time
        total_elapsed_time = time.time() - start_time

        # Calculate estimated time remaining
        avg_time_per_chunk = total_elapsed_time / (chunk_idx + 1)
        total_chunks_estimate = sum(1 for _ in open(input_file_path)) // chunksize + 1
        estimated_time_remaining = avg_time_per_chunk * (total_chunks_estimate - (chunk_idx + 1))

        # Print time tracking information
        print(f"Processed chunk {chunk_idx + 1}/{total_chunks_estimate}")
        print(f"Time elapsed for chunk: {chunk_elapsed_time:.2f} seconds")
        print(f"Total time elapsed: {total_elapsed_time:.2f} seconds")
        print(f"Estimated time remaining: {estimated_time_remaining / 60:.2f} minutes")

    except Exception as e:
        print(f"Exception found in chunk {chunk_idx}: {str(e)}")
        pass

# Final output
print(f"Total number of patients identified: {total_patients_identified}")


In [ ]:
# Step 2: Process for pregnancy_corrected.csv
warnings.filterwarnings('ignore')

chunksize = 500000  # Adjust the chunk size as needed

# Initialize variables for tracking
complete_info_patients = 0
incomplete_info_patients = 0
start_time = time.time()  # Start time for tracking

# Calculate the total number of rows for estimating total chunks
total_rows = sum(1 for _ in open(output_file_fixed))  # Total number of rows
total_chunks = total_rows // chunksize + 1  # Calculate total chunks

# Process for pregnancy_corrected.csv
for chunk_idx, chunk_df in enumerate(pd.read_csv(output_file_fixed, dtype=str, chunksize=chunksize)):

    # Convert date columns to datetime format
    chunk_df['from_dt'] = pd.to_datetime(chunk_df['from_dt'], errors='coerce')
    chunk_df['to_dt'] = pd.to_datetime(chunk_df['to_dt'], errors='coerce')

    # Filter out rows with invalid dates
    chunk_df = chunk_df[chunk_df['from_dt'].notnull() & chunk_df['to_dt'].notnull()]

    # Calculate total pregnancies
    chunk_df = calculate_total_pregnancies(chunk_df, delivery_codes, 'pat_id', 'from_dt', 'to_dt', 'total_pregnancies', 'dts_pregnancies')
    chunk_df.reset_index(drop=True, inplace=True)

    # Identify patients with incomplete pregnancy information
    chunk_df['is_complete_info'] = chunk_df.apply(
        lambda x: len(x['dts_pregnancies'].split(',')) == int(x['total_pregnancies']) if pd.notnull(x['dts_pregnancies']) else False, axis=1
    )

    complete_info_patients += chunk_df[chunk_df['is_complete_info'] == True].shape[0]
    incomplete_info_patients += chunk_df[chunk_df['is_complete_info'] == False].shape[0]

    # Save to pregnancy_corrected.csv (output_file_fixed)
    if chunk_idx == 0:
        chunk_df.to_csv(output_file_fixed, mode='w', index=False)
    else:
        chunk_df.to_csv(output_file_fixed, mode='a', index=False, header=False)

    # Time tracking and progress reporting
    elapsed_time = time.time() - start_time
    processed_chunks = chunk_idx + 1
    avg_time_per_chunk = elapsed_time / processed_chunks
    remaining_chunks = total_chunks - processed_chunks
    estimated_time_remaining = avg_time_per_chunk * remaining_chunks

    print(f"Processed chunk {processed_chunks}/{total_chunks}")
    print(f"Time elapsed: {elapsed_time:.2f} seconds")
    print(f"Estimated time remaining: {estimated_time_remaining / 60:.2f} minutes")

# Final output
print(f"Number of patients with complete pregnancy information: {complete_info_patients}")
print(f"Number of patients with incomplete pregnancy information: {incomplete_info_patients}")


In [ ]:
# Step 3: Process for pregnancy_processed.csv
chunksize = 500000
total_patients_processed = 0
total_pregnancies = 0

# Determine if the output file already exists and load processed patients if it does
if os.path.exists(output_file_final):
    processed_df = pd.read_csv(output_file_final, usecols=['pat_id'])
    total_patients_processed = processed_df['pat_id'].nunique()
    print(f"Resuming from last processed point. Already processed patients: {total_patients_processed}")
    processed_patients = set(processed_df['pat_id'].unique())  # Get the list of already processed patient IDs
else:
    processed_patients = set()

# Calculate total rows and total chunks for estimation
total_rows = sum(1 for _ in open(output_file_fixed))
total_chunks = total_rows // chunksize

# Start time for tracking
start_time = time.time()  # Start time for tracking the entire process

# Read chunks from the corrected file and continue processing
for chunk_idx, chunk_df in enumerate(pd.read_csv(output_file_fixed, chunksize=chunksize)):
    # Track the start time of processing the current chunk
    chunk_start_time = time.time()

    # Filter out patients that are already processed
    chunk_df = chunk_df[~chunk_df['pat_id'].isin(processed_patients)]

    if chunk_df.empty:  # If the chunk is empty, skip
        continue

    # Only process patients with at least one pregnancy
    chunk_df = chunk_df[chunk_df['total_pregnancies'] != 0]

    # Process pregnancies for each patient
    chunk_df = process_pregnancies(chunk_df)
    chunk_df.reset_index(drop=True, inplace=True)

    # Count total patients and pregnancies in this chunk
    total_patients_processed += chunk_df['pat_id'].nunique()
    total_pregnancies += chunk_df[chunk_df['pat_id_p'].notnull()].shape[0]  # Count all non-null pregnancy identifiers

    # Save to pregnancy_processed.csv
    if os.path.exists(output_file_final):
        chunk_df.to_csv(output_file_final, mode='a', index=False, header=False)
    else:
        chunk_df.to_csv(output_file_final, mode='w', index=False)

    # Calculate time taken to process the current chunk
    chunk_elapsed_time = time.time() - chunk_start_time
    print(f"Chunk {chunk_idx + 1}/{total_chunks} processed in {chunk_elapsed_time:.2f} seconds.")

    # Calculate and display total progress and time estimation
    elapsed_time = time.time() - start_time
    processed_chunks = chunk_idx + 1
    avg_time_per_chunk = elapsed_time / processed_chunks
    remaining_chunks = total_chunks - processed_chunks
    estimated_time_remaining = avg_time_per_chunk * remaining_chunks

    print(f"Total processed: {processed_chunks}/{total_chunks} chunks.")
    print(f"Time elapsed: {elapsed_time:.2f} seconds. Estimated time remaining: {estimated_time_remaining:.2f} seconds.")

# Final output
print(f"Total number of patients processed: {total_patients_processed}")
print(f"Total number of pregnancies processed: {total_pregnancies}")